In [6]:

import time
import importlib
import math

import cv2 as cv
import numpy as np
import mediapipe as mp

import utils
importlib.reload(utils)

mp_face_mesh = mp.solutions.face_mesh

In [7]:
# landmark detection function 
def landmarksDetection(img, results, draw=False):
    img_height, img_width= img.shape[:2]
    # list[(x,y), (x,y)....]
    mesh_coord = [(int(point.x * img_width), int(point.y * img_height)) for point in results.multi_face_landmarks[0].landmark]
    if draw :
        [cv.circle(img, p, 2, utils.GREEN, -1) for p in mesh_coord]

    # returning the list of tuples for each landmarks 
    return mesh_coord

In [8]:
# face bounder indices 
FACE_OVAL=[ 10, 338, 297, 332, 284, 251, 389, 356, 454, 323, 361, 288, 397, 365, 379, 378, 400, 377, 152, 148, 176, 149, 150, 136, 172, 58, 132, 93, 234, 127, 162, 21, 54, 103,67, 109]

# lips indices for Landmarks
LIPS=[ 61, 146, 91, 181, 84, 17, 314, 405, 321, 375,291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95,185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78 ]
LOWER_LIPS =[61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
UPPER_LIPS=[ 185, 40, 39, 37,0 ,267 ,269 ,270 ,409, 415, 310, 311, 312, 13, 82, 81, 42, 183, 78] 
# Left eyes indices 
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
LEFT_EYEBROW =[ 336, 296, 334, 293, 300, 276, 283, 282, 295, 285 ]

# right eyes indices
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ]  
RIGHT_EYEBROW=[ 70, 63, 105, 66, 107, 55, 65, 52, 53, 46 ]

In [9]:
# Connect to cam
cap = cv.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1, 
    refine_landmarks=True, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Show image
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_height, img_width = frame.shape[:2]   # This is the actual measurements of the frame size.  We'll use this to multiply by the normalised x,y coordinates from results.multi_face_landmarks
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_coords = landmarksDetection(frame, results, False)
            #print(mesh_coords[p] for p in RIGHT_EYE)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in RIGHT_EYE], utils.GREEN, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in LEFT_EYE], utils.GREEN, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in RIGHT_EYEBROW], utils.RED, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in LEFT_EYEBROW], utils.RED, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in LOWER_LIPS], utils.PINK, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in UPPER_LIPS], utils.PINK, opacity=0.5)
            frame = utils.fillPolyTrans(frame, [mesh_coords[p] for p in FACE_OVAL], utils.GRAY, opacity=0.1)

        cv.imshow('Webcam', frame)

        # check if any keys are being pressed, if this is 'q' break out of while loop and release webcam & close frame
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()

## Blink calculator 

In [15]:
def euclaideanDistance(point, point1):
    x, y = point 
    x1, y1 = point1
    distance = math.sqrt((x1-x)**2 + (y1-y)**2)
    return distance 

def blinkRatio(img, landmarks, right_indices, left_indices):
    # Right eye
    # horizontal line
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]

    cv.line(frame, rh_right, rh_left, utils.GREEN, 2)
    cv.line(frame, rv_top, rv_bottom, utils.GREEN, 2)


In [16]:
# Connect to cam
cap = cv.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1, 
    refine_landmarks=True, 
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Show image
        frame = cv.flip(frame, 1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_height, img_width = frame.shape[:2]   # This is the actual measurements of the frame size.  We'll use this to multiply by the normalised x,y coordinates from results.multi_face_landmarks
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            mesh_coords = landmarksDetection(frame, results, False)
            #print(mesh_coords[p] for p in RIGHT_EYE)
            blinkRatio(frame, mesh_coords, RIGHT_EYE, LEFT_EYE)

        cv.imshow('Webcam', frame)

        # check if any keys are being pressed, if this is 'q' break out of while loop and release webcam & close frame
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
cap.release()
cv.destroyAllWindows()